In [38]:
#effigy sync process

import os
from datetime import datetime
from PIL import Image 
import glob
import ntpath

import mysql.connector
from mysql.connector import errorcode
import logging


logging.basicConfig(level=logging.DEBUG, filename='effigySync2.log', format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)


In [39]:

#
SOURCEPATH='Y:/server/EFFIGY_PICS'
#SOURCEPATH='Y:/server/EFFIGY_PICS2'
SOURCEPATH = 'C:/Users/user/Pictures'
MISSING=''
MISSING_0='0'
MISSING_0='0'
MISSING_DT='2100-01-01 0:0:0'
MIN_IMAGE_SIZE = 4000

In [40]:
def ConvertListToDic(lst): 
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)} 
    return res_dct 

In [41]:
def fixSlashes(val):
    val = val.replace('\\', '/')
    return val   


In [42]:
def getDt():
    now = datetime.now()
    dt = now.strftime('%Y-%m-%d %H:%M:%S')
    return(dt)


In [43]:
getDt()

'2020-04-16 00:16:08'

In [44]:

a=set()
#folders = []
# r=root, d=directories, f = files
for r, d, f in os.walk(SOURCEPATH):       
    for f1 in f:
        ext = os.path.splitext(f1)[1]
        a.add(ext) 

In [45]:
def createThumbnailss(srcPath,destPath,size=120):
    for infile in glob.glob(srcPath):
        print("infile",infile)
        file, ext = os.path.splitext(infile)
        print(file,ext)
        print(destPath)
        im = Image.open(infile)
        im.thumbnail(size,size)
        im.save(destPath + "/thumbnail1.png", "PNG")

In [46]:
def createThumbnails(srcFile, destPath, basewidth = 30):
    img = Image.open(srcFile)
    file = ntpath.basename(srcFile)

    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    destFile =  "./"+ file
    img.save(destFile)
    

In [47]:
createThumbnails("Y:\server\EFFIGY_PICS2\Zdjecia\other/IMG_0132.jpg","c:/temp",)

In [48]:
def getID(tbl,thisid, key,val):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (val,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + str(val)
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ str(val))
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [49]:
def getImageID(tbl,thisid, key,val,key2,val2):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s and  " + key2 + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (fixSlashes(val),val2,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [50]:
newid= getImageID("media","media_id", "path","c:\\","name","test.jpg")

[]


In [51]:
def getValidExtensions():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select extension from supported_ext order by 1"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        ext_set=set()  
        for i in rec:
            ext_set.add (i[0])
        return (ext_set)

In [52]:
def getValidMediType_Extensions():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select extension, media_type_id from media_type_ext order by 1"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        return (dict(rec))    

In [53]:
r = getValidMediType_Extensions()
r.get('.avi')

'V'

In [54]:
def getSkipFolder():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select skip_root from skip_root"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        _set=set()  
        for i in rec:
            _set.add (i[0])
        return (_set)

In [55]:
getSkipFolder()

{'Y:\\server\\EFFIGY_PICS2\\Zdjecia\\tomcat-ZDJECIA',
 'Y:\\server\\EFFIGY_PICS\\Zdjecia\\tomcat-ZDJECIA'}

In [56]:
import functools 
import operator  
def convertTuple(tup): 
    str1 =  ''.join(str(tup)) 
    return str1

In [57]:
#get valid extensions
EXT = getValidExtensions()
#  check
'avi' in EXT
EXT

{'.avi', '.jpeg', '.jpg', '.mov', '.mp3', '.mp4', '.mpg', '.png', '.wmv'}

In [58]:
def insertRoot (root):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_root values (null, %s)"
        root = fixSlashes(root)
        #print (root)
        mycursor.execute(ssql, (root,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_root","root_id", "root",fixSlashes(root))
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Root-Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Root - DB Closed")
        return (newid)

In [59]:
getID("loc_folder","folder_id", "folder_path","C:ggg/Uffsssehhffrsggs/user/Pictures/2017-11111111111")

[]


-1

In [60]:
def insertFolder (root_id,root,folder,folderPath):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_folder values (null,%s, %s,%s,%s)"
        root = fixSlashes(root)
        folder = fixSlashes(folder)
        folderPath = fixSlashes(folderPath)
        #print (ssql)
        mycursor.execute(ssql, (root_id,root,folder,folderPath,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_folder","folder_id", "folder_path",fixSlashes(folderPath))
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [61]:
def skipRoot(root):
    skip=False
    toSkip="Y:\server\EFFIGY_PICS\Zdjecia\\tomcat-ZDJECIA"
    if len(root) >= len(toSkip):
        if (fixSlashes(toSkip).upper() == fixSlashes(root[:len(toSkip)]).upper())  :
            skip=True
    #else:
    #    print("not matching length")
    return(skip)

In [62]:
def skipFolder(root):
    skip=False
    toSkip=getSkipFolder()
    for folder in toSkip:
        if len(root) >= len(folder):
            #print(folder)
            #print(fixSlashes(folder).upper())
            #print(fixSlashes(root[:len(folder)]).upper()) 
            if (fixSlashes(folder).upper() == fixSlashes(root[:len(folder)]).upper())  :
                skip=True
                return(skip)
    return(skip)
        


In [63]:
skipFolder("Y:\server\EFFIGY_PICS2/Zdjecia\\tomcat-ZDJECIA")

True

In [64]:
#newid = insertFolder(1, "C:/Users/user/Pictures","2017-1111", "C:ggg/Uffsssehhffrsggs/user/Pictures/2017-1221111111111")
#newid

In [65]:
def insertImage (key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):
    logger.info("Saving image")
    #print(key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into media values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        key = fixSlashes(key)
        path = fixSlashes(path)
        parent_folder = fixSlashes(parent_folder)
        #print (ssql)
        mycursor.execute(ssql, ( key,file_name,media_type,folder_id,path,parent_folder_id,parent_folder,"sDesc",file_name,0,ext,getDt(),getDt()))
        mydb.commit()
        #get newly created ID
        newid= getImageID("media","media_id", "path",path,"name",file_name)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [66]:
def importFoldersAndImages ():
    #get Valid extensions first
    c=0
    EXT = getValidExtensions()
    EXTTypeDic  = getValidMediType_Extensions()
    #  check
    '' in EXT
    #print(EXT)
    for r, d, f in os.walk(SOURCEPATH):
        #print("dir",d,r)
        if len(d) >0:
            if  skipRoot(r):
                print("Skipped Root:",r)
            else:
                idR = insertRoot(r)
                #print("Processed Root:",idR,r)
                for dir in d:
                    #print ("will INSERT dir:",idR,dir)
                    #print("about to insert", idR,r,dir,r + '/' + dir)
                    idF = insertFolder(idR,r,dir,r + "/"+ dir)
                    #print("Created folder",idF)
                    pth = r +"/"+dir
                    #print("path", pth)
                    files = [f for f in os.listdir(pth) if os.path.isfile(os.path.join(pth,f))]
                    for f in files:
                        file_ext= os.path.splitext(f)[1].lower()
                        key=r +"/"+dir + "/" + f
                        size = os.path.getsize(key)
                        #print("size", size,file_ext)
                        if (file_ext in EXT) and size > MIN_IMAGE_SIZE:
                            mediaType = EXTTypeDic.get(file_ext)
                            #print("Processing:" + f)
                            #inserImage(file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):                       
                            idM = insertImage(key,f,mediaType,idF, r +"/"+dir, idR,r,"",file_ext)
                            #print(idM)
                           
    return(c)                    
                    
    
                    
                    
                
        
    

In [67]:
c= importFoldersAndImages()
print ("total:",c)

[(77,)]
[(398,)]
[(32634,)]
[(32635,)]
[(32636,)]
[(32637,)]
[(32638,)]
[(32639,)]
[(32640,)]
[(32641,)]
[(32642,)]
[(399,)]
[(32643,)]
[(32644,)]
[(32645,)]
[(400,)]
[(32646,)]
[(32647,)]
[(32648,)]
[(32649,)]
[(32650,)]
[(32651,)]
[(32652,)]
[(32653,)]
[(32654,)]
[(32655,)]
[(32656,)]
[(32657,)]
[(32658,)]
[(32659,)]
[(32660,)]
[(32661,)]
[(32662,)]
[(32663,)]
[(32664,)]
[(32665,)]
[(32666,)]
[(32667,)]
[(32668,)]
[(32669,)]
[(32670,)]
[(32671,)]
[(401,)]
[(32672,)]
[(32673,)]
[(32674,)]
[(32675,)]
[(32676,)]
[(32677,)]
[(32678,)]
[(32679,)]
[(32680,)]
[(32681,)]
[(32682,)]
[(32683,)]
[(32684,)]
[(32685,)]
[(32686,)]
[(32687,)]
[(32688,)]
[(32689,)]
[(32690,)]
[(32691,)]
[(32692,)]
[(32693,)]
[(32694,)]
[(32695,)]
[(32696,)]
[(32697,)]
[(32698,)]
[(32699,)]
[(32700,)]
[(32701,)]
[(32702,)]
[(32703,)]
[(32704,)]
[(32705,)]
[(32706,)]
[(32707,)]
[(32708,)]
[(32709,)]
[(32710,)]
[(32711,)]
[(32712,)]
[(32713,)]
[(32714,)]
[(32715,)]
[(32716,)]
[(32717,)]
[(32718,)]
[(32719,)]
[(32720,)]

In [68]:
def getImages():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select media_id,path, name, media_key,media_type_id from media"
        #print (ssql)
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        _set=set()  
        for i in rec:
            _set.add (i)
        return (_set)

In [69]:
def getImageExifAttributes(pth,ext):
    exif= {}
    ex=""
    dic={}
    #check if image
    if ext == 'P':
        # Read image 
     try:   
        img = Image.open(pth) 
        dic = img._getexif()
        #check if empty
     except Exception as err:
        print("issue processing" + pth)
        print(err)
        logger.error(err)
        print("WIDTH",pth)
    if  dic:
        
        if str(dic.get(40963)).isnumeric():
            exif["WIDTH"] =  str(dic.get(40963))
        else:
            exif["WIDTH"] =  str('0')
            
        exif["HEIGHT"] = str( dic.get(40962))
        exif["CAMERA"] = str( dic.get(271))
        exif["MODEL"] =  str(dic.get(272))
        exif["DTTAKEN"] = str( dic.get(36868))
        exif["DTMODIFY"] = str( dic.get(36867))
        exif["FSTOP"] =  str(dic.get(42036))
        exif["EXPOSURE"] =  str(dic.get(40960))
        exif["ORIENTATION"] = str( dic.get(274))
        exif["FLASH"] =  str(dic.get(7385))
        exif["SHUTTER"] =  str(dic.get(37377))
        exif["APERTURE"] = str( dic.get(37378))
        exif["ISOSPEED"] = str( dic.get(34855))
        exif["DIGIZOOM"] =  str(dic.get(41988))
        #print("FSTOP",exif["FSTOP"]) 
        for x in dic:
             if x != 37500 and x != 37510:
                ex= ex + (str(x) +  ":" +  str(dic.get(x)) + chr(13) +chr(10) ) 
        #limit to 5K
        exif["EXIF"] =  str(ex)[0:5000] 
    else:
        exif["WIDTH"] =  MISSING_0
        exif["HEIGHT"] = MISSING_0
        exif["CAMERA"] = MISSING
        exif["MODEL"] =  MISSING
        exif["DTTAKEN"] = MISSING_DT
        exif["DTMODIFY"] = MISSING_DT
        exif["FSTOP"] =  MISSING
        exif["EXPOSURE"] =  MISSING
        exif["ORIENTATION"] = MISSING
        exif["FLASH"] = MISSING
        exif["SHUTTER"] =  MISSING
        exif["APERTURE"] = MISSING
        exif["ISOSPEED"] = MISSING
        exif["DIGIZOOM"] =  MISSING
        exif["EXIF"] =  MISSING 
    return(exif)

In [70]:
getImageExifAttributes("Y:/server/EFFIGY_PICS2/Zdjecia/IMG_0132noExif.jpg",'P')

{'WIDTH': '0',
 'HEIGHT': '0',
 'CAMERA': '',
 'MODEL': '',
 'DTTAKEN': '2100-01-01 0:0:0',
 'DTMODIFY': '2100-01-01 0:0:0',
 'FSTOP': '',
 'EXPOSURE': '',
 'ORIENTATION': '',
 'FLASH': '',
 'SHUTTER': '',
 'APERTURE': '',
 'ISOSPEED': '',
 'DIGIZOOM': '',
 'EXIF': ''}

In [71]:
size = os.path.getsize("Y:/server/EFFIGY_PICS2/Zdjecia/IMG_0132noExif.jpg")
print("size", size)

size 919


In [72]:
def insertMediaAttributes (width, height, camera, model, dttaken, dtmodify,\
                          fstop, exposure, orientation, flash, shutter, aperture,\
                          isospeed,  digizoom,  exif,  media_id ,media_key):
    logger.info("Saving Attributes" + media_key)
    #print(key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into attrib values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        #print (ssql)
        mycursor.execute(ssql, ( width, height, camera, model, dttaken, dtmodify,\
                          fstop,  exposure, orientation, flash, shutter, aperture,\
                          isospeed, digizoom, exif, media_id, media_key, getDt()))
        mydb.commit()
        #get newly created ID
        newid= getID("attrib","attrib_id", "media_id",media_id)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [73]:
def ProcessAttributes():
    img_set = getImages()
    for media_id,path,fname, media_key, media_type_id in img_set:
        pth = fixSlashes(path +"/"+fname)
        #print(pth)
        if media_type_id == 'P':
            exif=getImageExifAttributes(pth,media_type_id)
            size = os.path.getsize(pth)
            #print("size", size)
            if size > MIN_IMAGE_SIZE:
                idA = insertMediaAttributes(exif["WIDTH"],exif["HEIGHT"],exif["CAMERA"],exif["MODEL"],exif["DTTAKEN"],exif["DTMODIFY"],exif["FSTOP"],exif["EXPOSURE"],exif["ORIENTATION"],exif["FLASH"],exif["SHUTTER"],exif["APERTURE"],exif["ISOSPEED"],exif["DIGIZOOM"],exif["EXIF"], media_id ,media_key)
            #attrib_id,  width, height,  camera, model,  dttaken,  dtmodify,  fstop,  exposure,  orientation,  flash,  shutter,  aperture,  isospeed,  digizoom,  exif,  media_id ,media_key,'


In [74]:
ProcessAttributes()

[(22319,)]
[(22320,)]
[(22321,)]
1292 (22007): Incorrect datetime value: 'None' for column 'DTTAKEN' at row 1
[(22322,)]
[(22323,)]
[(22324,)]
[(22325,)]
[(22326,)]
[(22327,)]
[(22328,)]
[(22329,)]
[(22330,)]
[(22331,)]
[(22332,)]
[(22333,)]
[(22334,)]
[(22335,)]
[(22336,)]
[(22337,)]
[(22338,)]
[(22339,)]
[(22340,)]
[(22341,)]
[(22342,)]
[(22343,)]
[(22344,)]
[(22345,)]
[(22346,)]
1292 (22007): Incorrect datetime value: 'None' for column 'DTTAKEN' at row 1
[(22347,)]
1292 (22007): Incorrect datetime value: 'None' for column 'DTTAKEN' at row 1
[(22348,)]
[(22349,)]
[(22350,)]
[(22351,)]
[(22352,)]
[(22353,)]
[(22354,)]
[(22355,)]
[(22356,)]
[(22357,)]
[(22358,)]
[(22359,)]
[(22360,)]
[(22361,)]
[(22362,)]
[(22363,)]
[(22364,)]
[(22365,)]
[(22366,)]
[(22367,)]
[(22368,)]
[(22369,)]
[(22370,)]
[(22371,)]
[(22372,)]
[(22373,)]
[(22374,)]
[(22375,)]
[(22376,)]
[(22377,)]
[(22378,)]
[(22379,)]
[(22380,)]
[(22381,)]
[(22382,)]
[(22383,)]
[(22384,)]
[(22385,)]
[(22386,)]
[(22387,)]
[(22388,)]

In [75]:

img_set = getImages()
for media_id,path,fname, media_key, media_type_id in img_set:
    pth = fixSlashes(path +"/"+fname)
    print(pth)
    if media_type_id == 'P':
        exif=getImageExifAttributes(pth)
        print (len(exif["EXIF"]))

    

C:/Users/user/Pictures/2019 Adam Bierzmowanie/DSC00424.JPG


TypeError: getImageExifAttributes() missing 1 required positional argument: 'ext'

In [ ]:
exif = getImageExifAttributes("Y:/server/EFFIGY_PICS2\\Zdjecia/other/IMG_0025.jpg")
exif["WIDTH"]
#print(exif)

#for e in exif:
#    print(e)
print(str(exif["APERTURE"]))
print(exif["APERTURE"])
print(convertTuple(exif["APERTURE"]))
#idA = inserMediaAttributes(exif["WIDTH"],exif["HEIGHT"],exif["CAMERA"],exif["MODEL"],exif["DTTAKEN"],exif["DTMODIFY"],exif["FSTOP"],exif["EXPOSURE"],exif["ORIENTATION"],exif["FLASH"],exif["SHUTTER"],exif["APERTURE"],exif["ISOSPEED"],exif["DIGIZOOM"],"","1" ,"1")
             

In [ ]:
path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/test.jpg"
from PIL import Image
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 

# prints format of image 
print(img.format) 

# prints mode of image 
print(img.mode) 

In [ ]:

#path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/geotag.jpg"
path= "Y:/server/EFFIGY_PICS2/Zdjecia/other/IMG_0131.jpg"
# Python program to read 
# image using PIL module 
  
# importing PIL 
from PIL import Image 
  
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 
  
# prints format of image 
print(img.format) 
  
# prints mode of image 
print(img.mode) 
dic = img._getexif()
width = dic.get(40963)
height = dic.get(40962)
camera = dic.get(271)
model = dic.get(272)
dttaken = dic.get(36868)
dtmod= dic.get(36867)
fstop= dic.get(42036)
exposure = dic.get(40960) 
orientation=dic.get(274)
flash = dic.get(7385)
shutter = dic.get(37377)
apperture=dic.get(37378) 
isospeed=dic.get(34855) 
digzoom=dic.get(41988)
exif=""
